In [ ]:
import pandas as pd
import spacy
from collections import defaultdict


In [ ]:
peticao = " Em 02 de setembro de 2024 Ilmo. Sr. LEONILDO ROLIM DOS SANTOS (“Sr. Leonildo”)Ref: Parecer Jurídico Sigiloso (10/06/2024) e Proposta (16/08/2024)A/C.: Dr. Luis Bitetti – OAB/SP nº 84.009E-mail: bitetti@adv.oabsp.org.br  Prezado Senhor,	Na qualidade de advogados da BRL TRUST DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A., cessionária dos honorários contratuais então titulados pelo escritório AFFONSO FERREIRA ADVOGADOS, Dra. HALBA MERY PEREBONI ROCCO e Dr. HENRIQUE LINDENBOJM, em resposta ao parecer e à proposta em referência, e considerando que:(a) a ora Notificante adquiriu de boa-fé os créditos de honorários devidos por décadas de patrocínio da ação de Desapropriação Indireta nº 0527173-63.1994.4.03.6100, cujos requisitórios foram expedidos de forma apartada da verba principal;  b) Tal verba, de natureza alimentar (Súmula Vinculante 47/STF), é autônoma  e não possui relação com a indenização devida pela União em decorrência da desapropriação de áreas situadas no município de Ubatuba/SP (Ubatumirim) na década de setenta para a expansão de rodovia federal (BR-101); (c) Eventual confusão entre áreas desapropriadas, conforme alegação descrita no parecer, se daria exclusivamente por uma sobreposição de glebas dos Autores com outra limítrofe e titulada pela companhia AGRÍCOLA AREIA BRANCA. Aliás, conforme expressamente reconhecido na proposta de 16/08/2024; e (d) O transcurso de décadas sem oposição, a tempo e modo, pelo Notificado (Sr. Leonildo) ocorrendo, sem qualquer dúvida, a extinção de eventual direito pela ocorrência da prescrição, uma vez que a o trecho da estrada BR-101 reivindicado fora construído há mais de 50 (cinquenta) anos.Vimos notificá-lo da inconsistência da pretensão vertida no parecer e na proposição e da tentativa ilegítima da postulação frente à Notificante, a qual defenderá seu direito em qualquer via que se fizer necessária, inclusive na seara deontológica."
texto = "Em 02 de setembro de 2024, o Sr. Afranio Affonso Ferreira Neto solicitou que a empresa BRL Trust Distribuidora de Títulos e Valores Mobiliários S.A. fosse notificada no processo judicial nº 0527173-63.1994.4.03.6100. A correspondência foi enviada para o endereço Rua Hungria nº 888 - 5º andar, Jardim Europa, São Paulo - 01455-000, e um e-mail de confirmação foi encaminhado para advocacia@affonsoferreira.com.br. Para mais informações, o advogado responsável, registrado sob a OAB/SP nº 155.406, pode ser contatado pelo telefone 55 11 3813-9522."

In [ ]:
linhas = texto.split('. ')

# Inicializar a lista para armazenar as linhas da tabela
tabela = []

In [ ]:
training_data = []

doc = nlp(texto)
# Cria um dicionário temporário para armazenar os dados
temp_dict = {}
temp_dict['text'] = texto
temp_dict['entities'] = []


In [30]:
doc.ents

(Sr. Afranio Affonso Ferreira Neto,
 BRL,
 Valores Mobiliários S.A.,
 Rua Hungria,
 Jardim Europa,
 São Paulo,
 SP)

In [31]:
# Extrai as entidades
for ent in doc.ents:
    # Pega o início e o fim da entidade no texto
    start = ent.start_char
    end = ent.end_char
    # Transforma o label em letras maiúsculas para manter consistência
    label = ent.label_.upper()
    # Adiciona a entidade no formato (início, fim, label)
    temp_dict['entities'].append((start, end, label))

training_data.append(temp_dict)

In [32]:
# Imprime o primeiro elemento para verificar o resultado
print(training_data[0])

{'text': 'Em 02 de setembro de 2024, o Sr. Afranio Affonso Ferreira Neto solicitou que a empresa BRL Trust Distribuidora de Títulos e Valores Mobiliários S.A. fosse notificada no processo judicial nº 0527173-63.1994.4.03.6100. A correspondência foi enviada para o endereço Rua Hungria nº 888 - 5º andar, Jardim Europa, São Paulo - 01455-000, e um e-mail de confirmação foi encaminhado para advocacia@affonsoferreira.com.br. Para mais informações, o advogado responsável, registrado sob a OAB/SP nº 155.406, pode ser contatado pelo telefone 55 11 3813-9522.', 'entities': [(29, 62, 'PER'), (87, 90, 'ORG'), (124, 148, 'PER'), (263, 274, 'MISC'), (294, 307, 'LOC'), (309, 318, 'LOC'), (482, 484, 'LOC')]}


In [33]:
training_data[0]['text']

'Em 02 de setembro de 2024, o Sr. Afranio Affonso Ferreira Neto solicitou que a empresa BRL Trust Distribuidora de Títulos e Valores Mobiliários S.A. fosse notificada no processo judicial nº 0527173-63.1994.4.03.6100. A correspondência foi enviada para o endereço Rua Hungria nº 888 - 5º andar, Jardim Europa, São Paulo - 01455-000, e um e-mail de confirmação foi encaminhado para advocacia@affonsoferreira.com.br. Para mais informações, o advogado responsável, registrado sob a OAB/SP nº 155.406, pode ser contatado pelo telefone 55 11 3813-9522.'

In [34]:
training_data[0]['entities']

[(29, 62, 'PER'),
 (87, 90, 'ORG'),
 (124, 148, 'PER'),
 (263, 274, 'MISC'),
 (294, 307, 'LOC'),
 (309, 318, 'LOC'),
 (482, 484, 'LOC')]

In [35]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("pt")  # Carrega um modelo em branco
doc_bin = DocBin()  # Cria um objeto DocBin


In [36]:
from spacy.util import filter_spans

for training_example in tqdm(training_data):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start , end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_spans = filter_spans(ents)
    doc.ents = filtered_spans
    doc_bin.add(doc)
    
doc_bin.to_disk("train.spacy")
    

100%|██████████| 1/1 [00:00<00:00, 68.71it/s]
